In [1]:
#pip3 freeze > "requirements.txt"

import os
os.getcwd()
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
from scipy import stats
import pandas_gbq
import pingouin as pg

%run Question_Quality_Support_Function.ipynb

In [1]:
import pandas as pd

# create a sample table with MCQ number and submit time columns
data = {'MCQ': [1, 2, 3, 4, 5],
        'Submit Time': ['9:05', '9:16', '9:32', '9:47', '9:58']}
df = pd.DataFrame(data)

# calculate the time taken for each MCQ by subtracting time taken for previous MCQ from submit time of current MCQ
df['Time Taken'] = pd.to_datetime(df['Submit Time']) - pd.to_datetime(['9:00'] + df['Submit Time'].iloc[:-1].tolist())

# print the table with time taken column
print(df)


   MCQ Submit Time      Time Taken
0    1        9:05 0 days 00:05:00
1    2        9:16 0 days 00:11:00
2    3        9:32 0 days 00:16:00
3    4        9:47 0 days 00:15:00
4    5        9:58 0 days 00:11:00


In [2]:
pd.to_datetime(['9:00'] + df['Submit Time'].iloc[:-1].tolist())

DatetimeIndex(['2023-03-09 09:00:00', '2023-03-09 09:05:00',
               '2023-03-09 09:16:00', '2023-03-09 09:32:00',
               '2023-03-09 09:47:00'],
              dtype='datetime64[ns]', freq=None)

In [4]:
['9:00'] + df['Submit Time'].iloc[:-1].tolist()

['9:00', '9:05', '9:16', '9:32', '9:47']

In [2]:
file_path = 'queries/Question_data.sql'
file = open(file_path, "r")
question_data = pd.read_gbq(file.read())
question_data.head()

,problem_id,num_options,options,num_correct_option,correct_option,type_name
0,68,4,"209,211,210,212",1,212,Single choice
1,108,4,"372,370,369,371",1,371,Single choice
2,110,4,"379,380,378,377",1,377,Single choice
3,123,4,"429,430,431,432",1,432,Single choice
4,124,4,"433,434,436,435",1,435,Single choice


In [4]:
problem_list_choice = ['Java', 'SQL', 'Ruby on Rails']

# 'NodeJS','VueJS', 'Angular'
problem_list_choice


['NodeJS', 'VueJS', 'Angular', 'Java', 'SQL', 'Ruby on Rails']

In [5]:
for prb_name in problem_list_choice:
  print(f'-----------------------------------------------',
        f'\n        Running Loop for {prb_name}',
        '\n-----------------------------------------------')
  file_path = 'queries/submissions.sql'
  file = open(file_path, "r")
  submissions_query = file.read()   
  df = pd.read_gbq(submissions_query.replace('%s', prb_name))
  #df.to_csv('df.csv')
  #df = pd.read_csv('df.csv', index_col=0)
  df.dropna(inplace=True)
  df['answer'] = df['answer'].astype('str')
  df['answer_set'] = df['answer'].apply(lambda x:set(x.split(',')))

  # Deleting problem IDs where we have less than 200 submissions
  print(f'Submissions data has shape {df.shape}')
  df_low_attemp_prb = df.groupby('problem_id').agg({'developer_id':'nunique'}).reset_index()
  print(f'Toatl Problem for test {prb_name} are {df_low_attemp_prb.shape[0]}')
  df = df.loc[~df['problem_id'].isin(df_low_attemp_prb.loc[df_low_attemp_prb['developer_id']<200, 'problem_id'].values),].copy()
  print(f'After filtering submissions data has shape {df.shape}')

  df_pr = df.groupby(['problem_id', 'version']).agg({'developer_id':'count', 'score_by_answer':'mean'}).reset_index()
  df_pr2 = df_pr.merge(question_data, how='left', on='problem_id')

  df_pr2['options_set'] = df_pr2['options'].apply(lambda x:set(x.split(',')))
  df_pr2['correct_option_set'] = df_pr2['correct_option'].apply(lambda x:set(x.split(',')))

  df_pr2['options_set_without_answer'] = ''
  for ind in df_pr2.index:
    df_pr2.at[ind,'options_set_without_answer'] = df_pr2.at[ind, 'options_set'].difference(df_pr2.at[ind,'correct_option_set'])

  df_pr2.dropna(inplace=True)

  dict_all = {}
  dict_use = {}
  for ind in df_pr2.index:
    dict_sub = {}
    for j in df_pr2.at[ind, 'options_set_without_answer']:
      prb_id = df_pr2.at[ind, 'problem_id']
      tot_att = df_pr2.at[ind, 'developer_id']
      dict_sub[j] = sum(df.loc[df['problem_id']==prb_id, 'answer_set'].apply(lambda x:j in x))/tot_att
      dict_use[prb_id] = len([val for val in dict_sub.values() if val > 0.05])
      dict_all[prb_id] = dict_sub

  
  dis_df = pd.DataFrame.from_dict(dict_use,orient='index', columns=['distractor']).reset_index()
  dis_df.rename(columns={'index':'problem_id'}, inplace=True)
  dis_df_final = df_pr2.merge(dis_df, how='left', on='problem_id')
  dis_df_final['distractor_eff'] = round(100*(dis_df_final['distractor'])/(dis_df_final['num_options']-dis_df_final['num_correct_option']),2)
  dis_df_final.rename(columns = {'problem_id':'Problem ID', 'distractor_eff':'Distractor Efficiency (%)'}, inplace=True)
  dis_df_final.sort_values(by='Distractor Efficiency (%)', ascending=False).head()

  # check for Distractor Efficiency (%)
  #df[df['problem_id']==3992].answer.value_counts(normalize=True)

  dif = df.groupby(['version', 'problem_id', 'challenge_name', 'challenge_id']).agg({'submit_id' :'count', 'score_by_answer':'mean', 'problem_submit_time':'min'}).reset_index()
  dif['score_by_answer'] = round(100*dif['score_by_answer'],2)
  dif.rename(columns = {'version':'Version', 'challenge_name':'MCQ Name', 'challenge_id':'MCQ ID',  'problem_id':'Problem ID', 'submit_id':'Attempts', 'score_by_answer':'Pass (%)', 'problem_submit_time':'Launch Date'}, inplace=True)
  dif.head()

  df_old = df.copy()
  df_cor = pd.DataFrame()
  prb_list = list(df_old.problem_id.unique())

  ind = 0
  for pr in prb_list:
    temp = df_old[df_old['problem_id']==pr].copy()
    cor_ans = df_old[df_old['problem_id']!=pr].groupby('developer_id').agg({'score_by_answer':'sum'}).reset_index()
    cor_ans.rename(columns = {'score_by_answer':'total_correct'}, inplace=True)
    temp2 = temp.merge(cor_ans, how='inner', on='developer_id')
    df_cor.at[ind, 'problem_id'] = pr
    corr_value = stats.pointbiserialr(temp2['score_by_answer'], temp2['total_correct'])
    df_cor.at[ind, 'correlation'] = corr_value[0]
    df_cor.at[ind, 'pvlaue'] = corr_value[1]
    ind = ind+1
  
  df_cor['correlation'] = round(df_cor['correlation'],3)
  df_cor['pvlaue'] = round(100*df_cor['pvlaue'],6)

  df_cor.rename(columns = {'problem_id':'Problem ID', 'correlation':'Point biserial correlation', 'pvlaue':'P_value (%)'}, inplace=True)
  df_cor.head()


  file_path = 'queries/dc_ir.sql'
  file = open(file_path, "r")
  dc_ir_query = file.read()
  file.close()
  dc_ir = pd.read_gbq(dc_ir_query.replace('%s', prb_name))
  
  print(f'DC IR data has shape {dc_ir.shape}')

  dc_ir_gr = dc_ir.groupby(['problem_id','score_by_answer']).agg({'si_date':'count', 'chosen_date':'count'}).reset_index()
  dc_ir_gr.rename(columns = {'si_date':'Number of Interviews', 'chosen_date':'Number of DC', 'problem_id':'Problem ID'}, inplace=True)
  rw_dic = {0:'Wrong', 1:'Right'}
  dc_ir_gr['score_by_answer'] = dc_ir_gr['score_by_answer'].map(rw_dic)
  dc_ir_piv = dc_ir_gr.pivot(index='Problem ID', columns='score_by_answer', values=['Number of Interviews', 'Number of DC']).reset_index()
  dc_ir_piv.fillna(0, inplace=True)

  dc_ir_piv[('DC/IR', 'Right')] = round(100*dc_ir_piv[('Number of DC', 'Right')]/dc_ir_piv[('Number of Interviews', 'Right')],4)
  dc_ir_piv[('DC/IR', 'Wrong')] = round(100*dc_ir_piv[('Number of DC', 'Wrong')]/dc_ir_piv[('Number of Interviews', 'Wrong')],4)

  dc_ir_piv.head(3)

  dc_ir_piv[[('Problem ID', ''), ('DC/IR','Right')]].reset_index()
  dc_ir_final = pd.DataFrame()
  dc_ir_final['Problem ID'] = dc_ir_piv[('Problem ID', '')]
  dc_ir_final['Number of Interviews'] = dc_ir_piv[('Number of Interviews', 'Right')] + dc_ir_piv[('Number of Interviews', 'Wrong')]
  dc_ir_final['Number of DC'] = dc_ir_piv[('Number of DC', 'Right')] + dc_ir_piv[('Number of DC', 'Wrong')]
  dc_ir_final['DC/IR Difference'] = round(dc_ir_piv[('DC/IR','Right')] - dc_ir_piv[('DC/IR','Wrong')],2)
  dc_ir_final.fillna(0, inplace=True)
  dc_ir_final.head(2)
  df_cor['Problem ID'] = df_cor['Problem ID'].astype(int)

  df_merge = dif.merge(df_cor, how='left', on='Problem ID')
  df_merge['Launch Date'] = pd.to_datetime(df_merge['Launch Date']).dt.date
  df_final = dc_ir_final.merge(df_merge, how='right', on='Problem ID').merge(dis_df_final[['Problem ID','Distractor Efficiency (%)', 'type_name', 'num_options']], how='left', on='Problem ID')


  df_final.columns = ['Problem_ID', 'IRs','DCs','DC_IR_Difference', 'Version','MCQ_Name',\
                      'MCQ_ID', 'Attempts','Pass_pct','Launch_Date','Point_biserial_correlation',\
                      'P_value','Distractor_Efficiency','Question_Type','Options']
  df_final['Launch_Date'] = pd.to_datetime(df_final['Launch_Date'])

  df_final['Updated_date_run'] = pd.Timestamp.now()
  df_final = problem_selection(df_final)

  df_temp = pd.DataFrame()
  for mcq_ver in df_final.Version.unique():
    df_final_old = df_final[df_final['Version']==mcq_ver].copy()
    old_problems = [int(val) for val in df_final_old['Problem_ID'].unique()]
    old_problems_with_no_issue = [int(val) for val in df_final_old.loc[df_final_old['Decision']!='Discard Problem', 'Problem_ID'].unique()]
    df_pivot_dev_old = df[df['problem_id'].isin(old_problems)].pivot(index='developer_id', columns = 'problem_id', values='score_by_answer').reset_index()
    cr_before = pg.cronbach_alpha(data=df_pivot_dev_old[old_problems], ci=.95)
    cr_after = pg.cronbach_alpha(data=df_pivot_dev_old[old_problems_with_no_issue], ci=.95)

    df_final_old['cr_before'] = cr_before[0]
    df_final_old['cr_before_CI'] = ''
    df_final_old['cr_before_CI'] = str(cr_before[1])
    df_final_old['cr_after'] = cr_after[0]
    df_final_old['cr_after_CI'] = ''
    df_final_old['cr_after_CI'] = str(cr_after[1])

    pr_lift_del = []
    pr_lif_kept = []
    for ind in df_final_old.index:
      prb = int(df_final_old.at[ind, 'Problem_ID'])
      cr_temp = pg.cronbach_alpha(data=df_pivot_dev_old[[val for val in old_problems if val!=prb]], ci=.95)
      df_final_old.at[ind, 'Lift_pct'] = round(100*(cr_temp[0]-cr_before[0])/cr_before[0],4)
      if (cr_temp[0]-cr_before[0]) > 0:  # cr_temp is after elmiminating a question, if cr_temp > cr_before
                                        # means cronbach_alpha has increased and we should remove question
                                        # lift is positive. How much increase after deleted.  
        pr_lift_del.append(prb)
      else:
        pr_lif_kept.append(prb)
    pr_lif_kept = list(dict.fromkeys(pr_lif_kept))
    old_pso_lift_cr = pg.cronbach_alpha(data=df_pivot_dev_old[pr_lif_kept], ci=.95)
    df_final_old['cr_after_suggested'] = old_pso_lift_cr[0]
    df_final_old['cr_after_suggested_CI'] = ''
    df_final_old['cr_after_suggested_CI'] = str(old_pso_lift_cr[1])
    df_temp = pd.concat([df_temp, df_final_old])
    
  df_temp = df_temp.astype(str)
  print(f'Final Data has shape {df_temp.shape}')
  pandas_gbq.to_gbq(df_temp, 'pdsa.question_quality', project_id='turing-dev-337819', if_exists='append')


----------------------------------------------- 
        Running Loop for NodeJS 
-----------------------------------------------
Submissions data has shape (1056927, 18)
Toatl Problem for test NodeJS are 93
After filtering submissions data has shape (1056927, 18)
DC IR data has shape (88609, 22)
Final Data has shape (126, 25)


100%|██████████| 1/1 [00:00<?, ?it/s]


----------------------------------------------- 
        Running Loop for VueJS 
-----------------------------------------------
Submissions data has shape (388767, 18)
Toatl Problem for test VueJS are 58
After filtering submissions data has shape (388767, 18)
DC IR data has shape (12298, 22)
Final Data has shape (58, 25)


100%|██████████| 1/1 [00:00<?, ?it/s]


----------------------------------------------- 
        Running Loop for Angular 
-----------------------------------------------
Submissions data has shape (774143, 18)
Toatl Problem for test Angular are 56
After filtering submissions data has shape (774143, 18)
DC IR data has shape (16083, 22)
Final Data has shape (101, 25)


100%|██████████| 1/1 [00:00<?, ?it/s]


----------------------------------------------- 
        Running Loop for Java 
-----------------------------------------------
Submissions data has shape (1736844, 18)
Toatl Problem for test Java are 65
After filtering submissions data has shape (1736844, 18)
DC IR data has shape (30243, 22)


AssertionError: At least two items are required.

In [18]:
df.head()

,developer_id,submit_id,mcq_submit_time,challenge_id,challenge_name,version,total_score_by_problem,total_problem,mcq_passed,mcq_percentage,n_rank,rn,correct_percentage_bin,problem_id,answer,score_by_answer,problem_submit_time,answer_set
0,2681,3625450,2023-02-03 04:51:49+00:00,380,ReactJS,V-2,23,30,1,76.67,5,24,76-100,5485,22516,0,2023-02-03 04:51:49+00:00,{22516}
1,12891,3499676,2023-01-16 19:01:01+00:00,380,ReactJS,V-2,24,30,1,80.00,5,2,76-100,5479,22486,1,2023-01-16 18:46:21+00:00,{22486}
2,13589,3676767,2023-02-09 19:35:09+00:00,380,ReactJS,V-2,23,30,1,76.67,5,15,76-100,5462,22398,1,2023-02-09 19:33:37+00:00,{22398}
3,13589,3676767,2023-02-09 19:35:09+00:00,380,ReactJS,V-2,23,30,1,76.67,5,23,76-100,5475,22467,1,2023-02-09 19:15:31+00:00,{22467}
4,13589,3676767,2023-02-09 19:35:09+00:00,380,ReactJS,V-2,23,30,1,76.67,5,30,76-100,5483,22504,1,2023-02-09 19:21:34+00:00,{22504}


In [11]:

df_temp.Version.value_counts()

df_temp.groupby('Version').agg({'Problem_ID':'nunique'})

,Problem_ID
Version,
V-1,45
V-2,31
V-3,18


In [14]:

df_final.Version.value_counts()

#df_temp
#df.head()

V-1    52
V-2    39
V-3    33
Name: Version, dtype: int64

In [ ]:
pd.read_gbq('CREATE TABLE `turing-dev-337819.pdsa.question_quality` (`Problem_ID` string, `IRs` string, `DCs` string,\
            `DC_IR_Difference` string, `Version` string, `MCQ_Name` string,\
            `MCQ_ID` string, `Attempts` string, `Pass_pct` string, `Launch_Date` string,\
            `Point_biserial_correlation` string, `P_value` string,\
            `Distractor_Efficiency` string, `Question_Type` string, `Options` string,\
            `Updated_date_run` string, `Decision` string, `Reason` string,\
            `cr_before` string, `cr_before_CI` string, `cr_after` string,\
            `cr_after_CI` string, `Lift_pct` string, `cr_after_suggested` string,\
            `cr_after_suggested_CI` string)')

In [ ]:
 #("today").strftime("%Y-%m-%d")
#df_final['Updated_date_run'] = pd.to_datetime(df_final['Updated_date_run'])


In [ ]:
#pd.read_gbq('Select * from `turing-dev-337819.pdsa.question_quality`')
#pd.read_gbq('drop table `turing-dev-337819.pdsa.question_quality`')

In [3]:
df_ch_ver = pd.DataFrame(columns=['challenge_id', 'challenge_name', 'challenge_version'])

details = {
    'challenge_name' : ['Python', 'Python', 'ReactJS', 'ReactJS','ReactJS',\
                      'NodeJS', 'NodeJS','Angular','Angular','Web Frontend Basics',\
                      'JavaScript','JavaScript', 'VueJS', 'React Native','HTML/CSS/Javascript',\
                      'Java','Java','Java', 'Java', 'Java', 'SQL', 'SQL'],
    'challenge_id' : [211, 378, 176, 380, 399,\
                      173, 397, 199, 409, 420,\
                      229, 419, 208, 138, 159,\
                      120, 131, 426, 427, 428, 128,411],
    'challenge_version' : ['Old', 'New', 'Old', 'New', 'New',
                           'Old', 'New', 'Old', 'New', 'New',\
                           'Old', 'New', 'Old', 'Old', 'Old',\
                           'Old', 'Old', 'New', 'New', 'New','Old', 'New']
}

df_ch_ver = pd.DataFrame(details)
df_ch_ver

,challenge_name,challenge_id,challenge_version
0,Python,211,Old
1,Python,378,New
2,ReactJS,176,Old
3,ReactJS,380,New
4,ReactJS,399,New
5,NodeJS,173,Old
6,NodeJS,397,New
7,Angular,199,Old
8,Angular,409,New
9,Web Frontend Basics,420,New


In [2]:
a = [211, 378, 176, 380, 399,\
                      173, 397, 199, 409, 420,\
                      229, 419, 208, 138, 159,\
                      120, 131, 426, 427, 428, 128,411]
min(a)

120

In [4]:
#pd.read_gbq('CREATE TABLE `turing-dev-337819.pdsa.challenge_version`\
#             (`challenge_id` INT64, `challenge_name` string, `challenge_version` string)')
pandas_gbq.to_gbq(df_ch_ver, 'pdsa.challenge_version', project_id='turing-dev-337819', if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


In [5]:
pd.read_gbq('Select * from `turing-dev-337819.pdsa.challenge_version`')

,challenge_name,challenge_id,challenge_version
0,Python,378,New
1,ReactJS,380,New
2,ReactJS,399,New
3,NodeJS,397,New
4,Angular,409,New
5,Web Frontend Basics,420,New
6,JavaScript,419,New
7,Java,426,New
8,Java,427,New
9,Java,428,New
